In [12]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [13]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

[]

In [14]:
from langchain_community.document_loaders import PyPDFLoader
pages = []
dir = '../data'
# for filename in os.listdir(dir):
#     docs = []
#     if filename.endswith(".pdf"):
#         loader = PyPDFLoader(os.path.join(dir, filename))
#         pages += loader.load()
#     break # too much time, only load one file
# pages
# pages = [pages[0]]


pages = []
loader = PyPDFLoader(os.path.join(dir, '個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf'))
pages += loader.load()
pages = [pages[8]]
pages

[Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page': 8}, page_content='                                                                回目錄  \n6 \n版次： 113年07月版                                                                                            修訂日期： 113.0 7.01 參、  首期保險費繳交方式及相關規定  \n一、  首期保險費 -採金融機構 轉帳作業 \n(一) 首期保險費採 金融機構轉帳作業 應注意事項  \n1. 首期保險費由保戶 約定於本公司 簽約之金融機構所開立之活期帳戶扣款 。 \n2. 申請金融機構轉帳繳交 首期保險費 ，需填寫「保險費暨保險單借款利息自動轉帳付款授權\n書(下稱授權書 )，繳款人為非保單關係人時 須檢附關係證明文件。 相關規定請參照 繳費方\n式及受理應檢附文件 。 \n3. 要保人須於授權書要保人欄位簽 章並與要保書 簽章相同，授權人須留存金融機構之印鑑或\n簽章樣式 。 \n4. 授權書須送予授權之金融機構核對 授權人身分證 統一編號 及印鑑/簽章，經核印成功 (且\n同意承保 )始得進行首期保險費 轉帳扣款作業 。核印、扣款時間 請參照各金融機構核印、\n扣款時間及相關規定 。 \n5. 數位帳戶 請至各銀行臨櫃留存印鑑、核印成功便能授權扣款 (土地銀行、合作金庫、第一\n銀行此三家銀行無法授權扣款 )。 \n6. 生效日認定：  \n(1) 保單生效日 認定依「生效日期之認定原則 」辦理。 \n(2) 因核印失敗、請款失敗或 新契約於核保中主動異動繳 費管道者，其生效日認定 依「新\n契約首期保險費繳費管道異動之 繳費日認定原則 」辦理。  \n7. 保費折扣： 以金融機構轉帳 繳交保險費， 保費折扣適用規定依各險種投保規則辦理。  \n8. 新契約投保時授權書 不得為空白授權， 應依是否取得保單號碼 填寫相關欄位 ，未填寫完\n整須重新檢附 授權書。 相關規定請參照 自動轉帳及信用卡付款授權

In [8]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [9]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
model_name = 'qwen2:72b-instruct-q8_0'
llm = OllamaFunctions(model=model_name)
llm

/storage/pyenv/versions/3.12.5/envs/ACP_LLM_312/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 0.4.0. An updated version of the class exists in the langchain-ollama package and should be used instead. To use it run `pip install -U langchain-ollama` and import as `from langchain_ollama import ChatOllama`.
  warn_deprecated(


OllamaFunctions(model='qwen2:72b-instruct-q8_0')

In [10]:
import sys
sys.path.append("..")
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph, max_thread=1)
graph_builder.graph_build(pages)
graph_builder.chunk_list

[{'chunk_id': 'c5334ce3-d175-44dd-ae5b-2d461de269d2',
  'chunk_doc': Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 1}, page_content='                                                                回目錄   6  版次： 113年07月版                                                                                            修訂日期： 113.0 7.01 參、  首期保險費繳交方式及相關規定   一、  首期保險費 -採金融機構 轉帳作業  (一) 首期保險費採 金融機構轉帳作業 應注意事項   1. 首期保險費由保戶 約定於本公司 簽約之金融機構所開立之活期帳戶扣款 ')},
 {'chunk_id': '58100d60-8838-45c2-88e5-7f5b7e600709',
  'chunk_doc': Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 1}, page_content='。  2. 申請金融機構轉帳繳交 首期保險費')},
 {'chunk_id': 'd7aa3968-c409-4f51-9d26-23f5c9bbb071',
  'chunk_doc': Document(metadata={'source': '../data/個人保險首續期繳費暨保單貸款還款息作業手冊(113年7月版)_業務通路版.pdf', 'page_number': 1}, page_content='，需填寫「保險費暨保險單借款利息自動轉帳付款授權 書(下稱授權書 )，繳款人為非保單關係人時 須檢附關係證明文件。 相關規定請參照 繳費方 式及受理應檢附文件 。  3. 要保人須於授權書要保人欄位簽 章並與要保書 簽章相同，授權人

In [11]:
graph_builder._get_combined_chunks(graph_builder.chunk_list)

[Document(metadata={'combined_chunk_ids': ['c5334ce3-d175-44dd-ae5b-2d461de269d2']}, page_content='                                                                回目錄   6  版次： 113年07月版                                                                                            修訂日期： 113.0 7.01 參、  首期保險費繳交方式及相關規定   一、  首期保險費 -採金融機構 轉帳作業  (一) 首期保險費採 金融機構轉帳作業 應注意事項   1. 首期保險費由保戶 約定於本公司 簽約之金融機構所開立之活期帳戶扣款 '),
 Document(metadata={'combined_chunk_ids': ['58100d60-8838-45c2-88e5-7f5b7e600709']}, page_content='。  2. 申請金融機構轉帳繳交 首期保險費'),
 Document(metadata={'combined_chunk_ids': ['d7aa3968-c409-4f51-9d26-23f5c9bbb071']}, page_content='，需填寫「保險費暨保險單借款利息自動轉帳付款授權 書(下稱授權書 )，繳款人為非保單關係人時 須檢附關係證明文件。 相關規定請參照 繳費方 式及受理應檢附文件 。  3. 要保人須於授權書要保人欄位簽 章並與要保書 簽章相同，授權人須留存金融機構之印鑑或 簽章樣式 。  4. 授權書須送予授權之金融機構核對 授權人身分證 統一編號 及印鑑/簽章'),
 Document(metadata={'combined_chunk_ids': ['8f09dbeb-ce0b-4264-9cdd-1b46337fc983']}, page_content='，經核印成功 (且 同意承保 )始得進行首期保險費 轉帳扣款作業 。核印、扣款時間 請參照各金融機構核印、 扣款時間及相關規定 。  5. 數位帳戶 請至各銀行臨櫃留存印鑑、核印成功便能授

In [12]:

graph_documents = graph_builder.get_graph_from_llm(llm, graph_builder.chunk_list, allowedNodes=[], allowedRelationship=[])
graph_documents

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [08:07<00:00, 97.53s/it]


[GraphDocument(nodes=[Node(id='首期保險費', type='Insurancefee', properties={'description': '首次需繳交的保險費用'}), Node(id='金融機構轉帳作業', type='Paymentmethod', properties={'description': '通過銀行或金融机构進行转账支付的作业方式'}), Node(id='保戶', type='Policyholder', properties={'description': '购买保险并持有保险合同的人'}), Node(id='本公司', type='Insurancecompany', properties={'description': '提供保险服务的公司'}), Node(id='活期帳戶扣款', type='Accountdeduction', properties={'description': '从客户的活期账户中自动扣除费用'})], relationships=[Relationship(source=Node(id='首期保險費', type='Insurancefee'), target=Node(id='金融機構轉帳作業', type='Paymentmethod'), type='PAID_BY'), Relationship(source=Node(id='保戶', type='Policyholder'), target=Node(id='活期帳戶扣款', type='Accountdeduction'), type='DEDUCTS_FROM'), Relationship(source=Node(id='本公司', type='Insurancecompany'), target=Node(id='金融機構轉帳作業', type='Paymentmethod'), type='USES_PAYMENT_METHOD')], source=Document(metadata={'combined_chunk_ids': ['c5334ce3-d175-44dd-ae5b-2d461de269d2']}, page_content='                               

In [13]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

In [14]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

{'graph_doc': GraphDocument(nodes=[Node(id='首期保險費', type='Insurancefee', properties={'description': '首次需繳交的保險費用'}), Node(id='金融機構轉帳作業', type='Paymentmethod', properties={'description': '通過銀行或金融机构進行转账支付的作业方式'}), Node(id='保戶', type='Policyholder', properties={'description': '购买保险并持有保险合同的人'}), Node(id='本公司', type='Insurancecompany', properties={'description': '提供保险服务的公司'}), Node(id='活期帳戶扣款', type='Accountdeduction', properties={'description': '从客户的活期账户中自动扣除费用'})], relationships=[Relationship(source=Node(id='首期保險費', type='Insurancefee'), target=Node(id='金融機構轉帳作業', type='Paymentmethod'), type='PAID_BY'), Relationship(source=Node(id='保戶', type='Policyholder'), target=Node(id='活期帳戶扣款', type='Accountdeduction'), type='DEDUCTS_FROM'), Relationship(source=Node(id='本公司', type='Insurancecompany'), target=Node(id='金融機構轉帳作業', type='Paymentmethod'), type='USES_PAYMENT_METHOD')], source=Document(metadata={'combined_chunk_ids': ['c5334ce3-d175-44dd-ae5b-2d461de269d2'], 'id': '17d2c39897926bf31a67eb1de9

In [15]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)

In [16]:
# from langchain_community.vectorstores import Neo4jVector

# from langchain_openai import AzureOpenAIEmbeddings

# embedding = AzureOpenAIEmbeddings(
#     model="text-embedding-ada-002",
#     azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
#     azure_deployment='text-embedding-ada-002',
#     openai_api_version='2023-05-15'
# )

# vector = Neo4jVector.from_existing_graph(
#     embedding,
#     node_label='__Chunk__',
#     text_node_properties=['content'],
#     embedding_node_property='embedding'
# )

In [18]:
from langchain_community.vectorstores import Neo4jVector
# ! pip3 install -U langchain-huggingface
import os
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
from langchain_huggingface import HuggingFaceEmbeddings
# Choose from https://huggingface.co/spaces/mteb/leaderboard
embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")
vector = Neo4jVector.from_existing_graph(
    embedding,
    node_label='__Chunk__',
    text_node_properties=['content'],
    embedding_node_property='embedding'
)